The objective is to convert a sophisticated SEIR code written in C++ to Julia, little by little. We will start with low level of structure: how to store a graph. 

The population of Leon county in 2010 had about 260,000 people, and this population lives on a graph. Each person lives in a home, and most go to school or to work. 

So need to define a graph and its connections. Here is the C++ code for the structure of a node, found in head.h:
// Nodes refer to people in no particular order
typedef struct Node {
  //Info
  int age;
  //Infection
  int state;
  int hospitalization;
  //Connections
  int k;
  int *v;
  float *w;
  int is_vacc;      // 0,1 whether vaccinated or not

  // individual transmissibility (could be a function of time)
  // Leaky Vaccine (partially effective all all vaccinated people)
  // Hard Vaccine: Those vaccinated are immediately R. (equivalent to beta=infinity)
  // Assume that the vaccine is immediately effective
  // beta_is = params.beta * (1.-vacc1_effectiveness)
  // beta_is = params.beta * (1.-vacc2_effectiveness)
  float beta_IS;
  float mu; // individual recovery

  float vacc_infect;  // Vaccine doses reduce infectiousness of others
  float vacc_suscept; // Vaccine doses increase my resistance to the virus
  // Times patient enters the Latent, Sympt Infected and Recovered states (GE)
  float t_L, t_IS, t_R, t_V1, t_V2;
  int ti_L;  // integer time for infection (enter the latent stage)
} Node;```

We will not worry about efficiency. The easiest approach is a mutable structure. By default, `struct` in Julia is immutable (MUST EXPLAIN THIS): 

In [ ]:
# Create a function
function timeloop(nodes) #::Vector{Nodes})
    n = 260000
    for i in 1:n
        nodes[i].t_R = 1.
    end
end

In [ ]:
mutable struct Node
  #Info
  age::Int64
  #Infection
  state::Int64
  hospitalization::Int64
  #Connections
  k::Int64;
  #int *v;
  v::Vector{Int64}

  #float *w  # Need a dynamic array to be instantiated later
  w::Vector{Float64}
  is_vacc::Int64      #0 ,1 whether vaccinated or not

  #=  Multiline comment
    
  // individual transmissibility (could be a function of time)
  // Leaky Vaccine (partially effective all all vaccinated people)
  // Hard Vaccine: Those vaccinated are immediately R. (equivalent to beta=infinity)
  // Assume that the vaccine is immediately effective
  // beta_is = params.beta * (1.-vacc1_effectiveness)
  // beta_is = params.beta * (1.-vacc2_effectiveness)
  =#
    
  beta_IS::Float64;
  mu::Float64; # individual recovery

  vacc_infect::Float64;  # Vaccine doses reduce infectiousness of others
  vacc_suscept::Float64; # Vaccine doses increase my resistance to the virus
  # Times patient enters the Latent, Sympt Infected and Recovered states (GE)
  t_L::Float64
  t_IS::Float64
  t_R::Float64
  t_V1::Float64
  t_V2::Float64
  ti_L::Int64;  # integer time for infection (enter the latent stage)
  Node() = new(0, 0, 0, 0, zeros(Int64,1), zeros(Float64,1), 0, 0.,0., 0.,0., 0.,0.,0.,0.,0.,0)
end

In [ ]:
zeros(Int64,2)

# Demonstration of why one should execute code within a function
## I first create a vector of n nodes (260,000, the approx number of inhabitants in Leon County
- I initialize the nodes. Notice a default constructor inside the structure. 
- Other specialized structures are created outside the struct. 
- I then will 

In [ ]:
a = Vector{Node}()
nodes = a
n = 260000
resize!(a,n); 
for i in 1:n
    a[i] = Node()
end

In [ ]:
a[10]  # Demonstrates the nodes are initialized properly. 

# I can also demonstrate what happens if the default constructor is not present: the elements are initialized randomly. 

In [ ]:
mutable struct Node_test1
  #Info
  age::Int64
  #Infection
  state::Int64
  hospitalization::Int64
  Node_test1() = new()
  Node_test1(age) = new(age, 0, 0)
  Node_test1(age,state) = new(age, state, 0)
end


### Docs suggest minimizing number of internal constructors

In [ ]:
@show Node_test1(72)
@show Node_test1(42, 78)
@show Node_test1()  # Uninitialized

In [ ]:
c = b.age  # age is unitialized. Dangerous!!!

In [ ]:
@btime for i in 1:n
    a[i].t_R = 1.
end

In [ ]:
@btime for i in 1:n
    a[i].t_R = 1.
end
# 19 Mbytes memory allocation

In [ ]:
a1 = zeros(Float64, n)
@benchmark for i in 1:n
    a1[i] = 1.
end

In [ ]:
a = Vector{Float64}()   # define an empty vector
@show size(a)
# C++ equivalent: a = new std::vector()
resize!(a, 10);
@show size(a)

In [ ]:
a1 = Vector{Float64} # Bad
@show size(a)
resize!(a1, 10)  # CANNOT BE RESIZED

In [ ]:
@benchmark timeloop(nodes)

In [ ]:
@btime timeloop(nodes)  # No memory allocation

In [ ]:
# Create a function
function timeloop(nodes::Vector{Node})
    n = 260000
    @inbounds for i in 1:n
        nodes[i].t_R = 1.
    end
end

In [ ]:
@benchmark timeloop(nodes)
# @inbounds had very little effect (because few arrays in the loop)

In [ ]:
size(nodes)

C++ Code
```C++
#include <vector>

int main()
{
    std::vector<float> aa; aa.resize(200);
    std::vector<float>* a = new std::vector<float>();
    a->resize(40);
    // Dynamic memory allocation the heap for a and aa
    // sizeof(aa): 24, sizeof(a): 8
    printf("size(a,aa)= %lu, %lu\n", sizeof(a), sizeof(aa));
}
```

In [ ]:
aa = Vector{Float64}()
@show size(aa)
resize!(aa, 100);
@show size(aa)  # measured in floats
@show typeof(aa)

In [ ]:
@show bb = Array{Float64,1}([10.3,20,30])
@show b1 = Array{Int64,2}([[10,3] [5,2]]) # note space separator
@show size(b1)
@show b2 = Array{Int64,1}([[10,3]; [5, 3]])  # note semi-colon separator
@show sizeof(b2)  # number bytes
@show b3 = [[2,3],[4,5]] # 1D array
@show size(b2)
@show size(b3)  # dimensions of array
@show cc = Vector{Float64}([10,30,40,50]);
# semi-colon to avoid printing last line

In [ ]:
@which resize!(a, 10)

In [ ]:
mutable struct xx2
    v::Vector{Float64}
end

In [ ]:
xx2_ = xx2(zeros(8))

In [ ]:
resize!(xx2_.v, 12)
@show xx2_;

In [ ]:
resize!(x1.v,10)

# Return to Nodes and SEIR model
## We duplicate all the structs on head.h . This might change later as we optmize and restructure the code

```c++
typedef struct List {
  int *v;
  int n;
  int cum[NAGE];
} List;

typedef struct TIMES {
    int id_from;
    int id_to;
    int state_from;
    int state_to;
} TIMES;

typedef struct Counts {
    int count_l_asymp;
    int count_l_symp;
    int count_l_presymp;
    int count_i_symp;
    int count_recov;
    int count_vacc1;
    int count_vacc2;
    int countS;
    int countL;
    int countIS;
    int countR;
    int countV1;
    int countV2;
    std::vector<int> cvacc1, cvacc2;
    std::vector<int> cS, cL, cIS, cR;
    std::vector<float> times;
} Counts;

//Parameters
typedef struct Params {
    int N, n_runs, parameters;
    float r, epsilon_asymptomatic, epsilon_symptomatic;
    float p, gammita, mu, delta, muH, muICU, k, beta_normal;
    float alpha[NAGE], xi[NAGE], beta[NCOMPARTMENTS];
    float dt;
    float vacc1_rate;    //  nb 1st vaccinations per day
    float vacc2_rate;    //  nb 2nd vaccinations per day
    float vacc1_eff; //  % of people for whom 1st shot of the vaccine works as expected
    float vacc2_eff; //  % of people for whom 2nd shot of the vaccine works as expected
    // By default the same as the effectiveness (effect on transmissibility)
    float vacc1_recov_eff; //  reduction in recovery time due to vaccine shot
    float vacc2_recov_eff; //  reduction in recovery time due to vaccine shot
    float dt_btw_vacc; // Time between vacc1 and vacc2
    int max_nb_avail_doses;  // maximum number of available doses
    int nb_doses;      // 1 or 2 depending on the vaccine or experiment peformed
    float beta_shape;  // shape parameter for infectivity profile
    float beta_scale;  // scale parameter for infectivity profile
    std::vector<double> betaISt;  // Time-dependent profile for beta. Hardcoded for now.
    float R0;  // Initial R0 in the absence of social behavioral due to COVID-2.
} Params;

//Spreading
typedef struct Lists {
    int n_active, index_node;
    List susceptible; // Not clear whether necessary
    List latent_asymptomatic, latent_symptomatic, infectious_asymptomatic, pre_symptomatic, infectious_symptomatic, home, hospital, icu, recovered, vacc1, vacc2;
    List new_latent_asymptomatic, new_latent_symptomatic, new_infectious_asymptomatic, new_pre_symptomatic, new_infectious_symptomatic, new_home, new_hospital, new_icu, new_recovered, new_vacc1, new_vacc2;
    // Added by GE
    std::vector<int> id_from, id_to, state_from, state_to; //, from_time, to_time;
    std::vector<float> from_time, to_time;
    std::vector<int> people_vaccinated;
    std::vector<int> permuted_nodes;  // randomized population
} Lists;

//Network
typedef struct Network {
    Node *node;
    int start_search; // node to start search from to vaccinate
} Network;

typedef struct Files {
    //Various
    float t;
    int it;  // integer count
    //int t;
    FILE *f_cum, *f_data;
    char f_cum_file[255], f_data_file[255];
    char name_cum[255], name_data[255];
    char data_folder[255], result_folder[255];
    char parameter_file[255];
    char node_file[255];
    char network_file[255];
    char vaccination_file[255];
} Files;
```

Of the above structures, the Lists structures are the most important as they store information about the graph. We will start with defining a Julia `struct` called List. In C++, the list looks as follows: 
```C++ typedef struct List {
  int *v;
  int n;
  int cum[NAGE];
} List;```

The Julia version looks as follows, for now: 

In [ ]:
# Once defined, a list cannot be redefined. The issue relates to the global 
# scope. The proper approach is to work within a new module. We will ignore this
# for now. 
mutable struct List1
    v::Vector{Int64}
    n::Int64
    #m::Int64    # Uncomment this line. The code no longer works. 
    # cum::Vector{Int64} # ages. Ignore for now
    List1() = new(zeros(Int64,1), 0, 0)
end

In [ ]:
#  The struct we will use
mutable struct List
    v::Vector{Int64}
    n::Int64
    #m::Int64    # Uncomment this line. The code no longer works. 
    # cum::Vector{Int64} # ages. Ignore for now
    List() = new(zeros(Int64,1),0)  # not allowed to redefine the constructor
end

In [ ]:
l = List()

## The next step is to read in a network, and construct some routines to add and delete from the list. 
- Here are the C++ codes

```C++
void G::addToList(List *list, int id)
{
  list->v[list->n] = id;
  list->n++;
}

void G::updateList(List* list, List *newl, Network& network)
{
  for(int i=0;i<newl->n;i++)
    {
      list->v[list->n] = newl->v[i];
      list->n++;
      list->cum[network.node[newl->v[i]].age]++;  // Ignore for now
    }
}

int G::removeFromList(List *list, int i)
{
  list->n--;
  list->v[i] = list->v[list->n];

  return i-1;
}
```


And we should also create initializationa routines

```C++
void G::resetVariables(Lists& l, Files& files)
{
  l.susceptible.n = 0;
  l.latent_asymptomatic.n = 0;
  l.latent_symptomatic.n = 0;
  l.infectious_asymptomatic.n = 0;
  l.pre_symptomatic.n = 0;
  l.infectious_symptomatic.n = 0;
  l.home.n = 0;
  l.hospital.n = 0;
  l.icu.n = 0;
  l.recovered.n = 0;
  l.vacc1.n = 0;
  l.vacc2.n = 0;

  for(int i=0; i < NAGE; i++)
  {
      l.susceptible.cum[i] = 0;
      l.latent_asymptomatic.cum[i] = 0;
      l.latent_symptomatic.cum[i] = 0;
      l.infectious_asymptomatic.cum[i] = 0;
      l.pre_symptomatic.cum[i] = 0;
      l.infectious_symptomatic.cum[i] = 0;
      l.home.cum[i] = 0;
      l.hospital.cum[i] = 0;
      l.icu.cum[i] = 0;
      l.recovered.cum[i] = 0;
      l.vacc1.cum[i] = 0;
      l.vacc2.cum[i] = 0;
  }

  files.t = 0;
  l.n_active = 0;

  l.id_from.resize(0);
  l.id_to.resize(0);
  l.state_from.resize(0);
  l.state_to.resize(0);
  l.from_time.resize(0);
  l.to_time.resize(0);

 // Transmission profile
  // We will use a Weibull Distribution with scale 5.665 and shape 2.826,
  // from the paper by Ferretti (2020), "Quantifying SARS-COV-2" transmission
  // suggests epidemic control with digital contact tracing."
  par.betaISt.resize(3000);
  double shape = 2.826;
  double scale = 5.665;
  double sum = 0.0;

  // 3000 is overkill, but it allows me to avoid an if statement in getBetISt()
  for (int i=0; i < 3000; i++) {
     double t = i * par.dt;
     par.betaISt[i] = gsl_ran_weibull_pdf(t, scale, shape);
     printf("beta: %f\n", par.betaISt[i]);
     sum += par.betaISt[i];
     printf("beta[%f]= %f\n", i*par.dt, par.betaISt[i]);
  }
  printf("integral of betaISt= %f\n", sum*par.dt);
}

void G::resetNodes(Params& par, Network& net)
{
    for(int i=0; i < par.N; i++) {
        net.node[i].state = S;
        net.node[i].is_vacc = 0;
        net.node[i].beta_IS = par.beta[IS];
        net.node[i].mu = par.mu;
        net.node[i].vacc_infect  = 1.0;
        net.node[i].vacc_suscept = 1.0;
    }

    net.start_search = 0;
}

void G::resetNew(Lists& l)
{
  l.new_latent_asymptomatic.n = 0;
  l.new_latent_symptomatic.n = 0;
  l.new_infectious_asymptomatic.n = 0;
  l.new_pre_symptomatic.n = 0;
  l.new_infectious_symptomatic.n = 0;
  l.new_home.n = 0;
  l.new_hospital.n = 0;
  l.new_icu.n = 0;
  l.new_recovered.n = 0;
  l.new_vacc1.n = 0;
  l.new_vacc2.n = 0;
}
```

# Let us start with addToList: 

```C++
void G::addToList(List *list, int id)
{
  list->v[list->n] = id;
  list->n++;
}```
In Julia, 

In [ ]:
# by typing the arguments, we tell the compiler the second argumetn will always be an Int64
# and the first argument will always be a list. Specialization will help the multiple dispatcher 
# in case of function overload. But decreases generality. Some argument types can be removed at a later
# time if needed.

function addToList(list::List, id::Int64)
    # n is initially zero, count from 1. I could initialize n to -1 but that is unnatural
    list.v[list.n+1] = id  
    list.n += 1    # The ++ operator does not exist
end

In [ ]:
list = List()
sizeof(list.v)

In [ ]:
addToList(list, 3)

- Clearly, I forgot to initialize list.v 
- This is done in a method called G::allocateMemory(...)

``` C++
void G::allocateMemory(Params& p, Lists& l)
{
  //Spreading
  // Memory: 4.4Mbytes for these lists for Leon County, including vaccines
  printf("memory: %lu bytes\n", p.N * sizeof(*l.latent_asymptomatic.v) * 11 * 2);
  printf("sizeof(int*)= %ld\n", sizeof(int*));
  l.susceptible.v = (int*) malloc(p.N * sizeof * l.susceptible.v);
  l.latent_asymptomatic.v = (int*) malloc(p.N * sizeof * l.latent_asymptomatic.v);
  l.latent_symptomatic.v = (int*) malloc(p.N * sizeof * l.latent_symptomatic.v);
  l.infectious_asymptomatic.v = (int*) malloc(p.N * sizeof * l.infectious_asymptomatic.v);
  l.pre_symptomatic.v = (int*) malloc(p.N * sizeof * l.pre_symptomatic.v);
  l.infectious_symptomatic.v = (int*) malloc(p.N * sizeof * l.infectious_symptomatic.v);
  l.home.v = (int*) malloc(p.N * sizeof * l.home.v);
  l.hospital.v = (int*) malloc(p.N * sizeof * l.hospital.v);
  l.icu.v = (int*) malloc(p.N * sizeof * l.icu.v);
  l.recovered.v = (int*) malloc(p.N * sizeof * l.recovered.v);
  l.vacc1.v = (int*) malloc(p.N * sizeof * l.vacc1.v);
  l.vacc2.v = (int*) malloc(p.N * sizeof * l.vacc2.v);

  //New spreading
  l.new_latent_asymptomatic.v = (int*) malloc(p.N * sizeof * l.new_latent_asymptomatic.v);
  l.new_latent_symptomatic.v = (int*) malloc(p.N * sizeof * l.new_latent_symptomatic.v);
  l.new_infectious_asymptomatic.v = (int*) malloc(p.N * sizeof * l.new_infectious_asymptomatic.v);
  l.new_pre_symptomatic.v = (int*) malloc(p.N * sizeof * l.new_pre_symptomatic.v);
  l.new_infectious_symptomatic.v = (int*) malloc(p.N * sizeof * l.new_infectious_symptomatic.v);
  l.new_home.v = (int*) malloc(p.N * sizeof * l.new_home.v);
  l.new_hospital.v = (int*) malloc(p.N * sizeof * l.new_hospital.v);
  l.new_icu.v = (int*) malloc(p.N * sizeof * l.new_icu.v);
  l.new_recovered.v = (int*) malloc(p.N * sizeof * l.new_recovered.v);
  l.new_vacc1.v = (int*) malloc(p.N * sizeof * l.new_vacc1.v);
  l.new_vacc2.v = (int*) malloc(p.N * sizeof * l.new_vacc2.v);
}```

`gordon` 
'''
What is done
'''

I will create a function to allocate memory for a single list

In [ ]:
function allocateMemory(list::List, N::Int32)
    # 16 bit int stores 65k values. Lists default to size N=260,000
    # So I need 32 bits. 
    list.v = zeros(Int32, N)
end

In [ ]:
list = List()
n = Int32(10)
allocateMemory(list, n);

In [ ]:
# Is this the best approach, or perhaps sub-structures would be better
# It is always about efficiency versus readability

# It is not clear whether I need a struct or whether I should pass the lists
# as need as argument. This approach seems cleaner.

mutable struct Lists
    n_active::Int64
    index_node::Int64
    
    susceptible::List
    latent_asymptomatic::List
    latent_symptomatic::List
    infectious_asymptomatic::List
    pre_symptomatic::List 
    infectious_symptomatic::List
    home::List
    hospital::List
    icu::List
    recovered::List
    vacc1::List
    vacc2::List
    new_latent_asymptomatic::List
    new_latent_symptomatic::List
    new_infectious_asymptomatic::List
    new_pre_symptomatic::List
    new_infectious_symptomatic::List
    new_home::List
    new_hospital::List 
    new_icu::List
    new_recovered::List
    new_vacc1::List
    new_vacc2::List

    id_from::Vector{Int64}
    id_to::Vector{Int64}
    state_from::Vector{Int64}
    state_to::Vector{Int64}
    from_time::Vector{Int64}
    to_time::Vector{Int64}
    people_vaccinated::Vector{Int64}
    permuted_nodes::Vector{Int64}
    
    # Ideally, I only wish to initialize the elements I will use
    Lists() = new(0,0, List(), List(), List(), List(), List(), List(), List(), List(), 
        List(), List(), List(), List(), List(), List(), List(), List(), 
        List(), List(), List(), List(), List(), List(), List(), 
        zeros(Int64,1), zeros(Int64,1), zeros(Int64,1), zeros(Int64,1),
        zeros(Int64,1), zeros(Int64,1), zeros(Int64,1), zeros(Int64,1))
end

In [ ]:
lists = Lists()

## There is probably a more julia-nic way to do this, but 
- First get the code running with the correct results (same as C++ code)
- Then use this as a benchmark to gauge refactorizations
- This is part of the software engineering process

## I will now initialize the lists I will require by create a routine allocateMemory with a different argument type: Lists  
- I used multiple dispatch with allocateMemory()

In [ ]:
function allocateMemory(l::Lists, N::Int32)
    # 16 bit int stores 65k values. Lists default to size N=260,000
    # So I need 32 bits. 
    allocateMemory(l.susceptible, N)
    allocateMemory(l.infectious_symptomatic, N)
    allocateMemory(l.recovered, N)
end

In [ ]:
N = Int32(260000)
lists = Lists()
allocateMemory(lists, N);

In [ ]:
methods(addToList)

In [ ]:
addToList(lists.recovered, 19)

In [ ]:
lists.recovered.n


## Apparently, addToLists works as expected. Next: 
- removeFromList()
- updateLists

```C++
void G::updateList(List* list, List *newl, Network& network)
{
  for(int i=0;i<newl->n;i++)
    {
      list->v[list->n] = newl->v[i];
      list->n++;
      list->cum[network.node[newl->v[i]].age]++;  // Ignore for now
    }
}

int G::removeFromList(List *list, int i)
{
  list->n--;
  list->v[i] = list->v[list->n];

  return i-1;
}
```

In [ ]:
function removeFromList(l::List, i::Int32)
    # Maintain a dense list by plugging the holes left by the node removed
    l.n -= 1
    l.v[i] = l.v[n]  # fill the ith slot with the nth node
    return i  # return the node removed from the list
end

In [ ]:
function updateList(l::List, newl::List, network::Network)
    for i in 1:newl.n
        l.v[l.n] = newl.v[i]
        l.n  += 1
        # I removed any age information
    end
end

## Next step: read the nodes and edges into the program and define the Network struct

In [ ]:
# edges will be encoded into the neighbor of each node in Nodes
mutable struct Network
    nodes::Vector{Node}
    start_search::Int64 # node to start search from to vaccinate
end